In [ ]:
# 将目标点云的变换反向应用到源点云上。
        inverse_transformation_target = np.linalg.inv(coarse_transformation_target)
        # 手动计算目标变换的逆
        # R = coarse_transformation_target[:3, :3]
        # t = coarse_transformation_target[:3, 3]
        # inverse_R = R.T
        # inverse_t = -np.dot(inverse_R, t)
        # inverse_transformation_target = np.eye(4)
        # inverse_transformation_target[:3, :3] = inverse_R
        # inverse_transformation_target[:3, 3] = inverse_t
        # 包含平移矢量的变换矩阵
        translation_matrix = np.eye(4)
        translation_matrix[:3, 3] = -translation_vector
        combined_coarse_transformation = np.dot(translation_matrix, np.dot(inverse_transformation_target, coarse_transformation_source))
        # 应用最终的变换矩阵到源点云
        self.source.transform(combined_coarse_transformation)
        # 可视化粗配准结果
        visualize_initial_point_clouds(self.source, self.target, window_name='针对source粗配准结果')
        fine_transformation = self.icp_fine_registration(self.source, self.target ,vs,  vt, self.threshold)

In [ ]:
def apply_pca(self, point_cloud):
        # 计算点云的主成分
        mean = np.mean(point_cloud.points, axis=0)
        point_cloud_centered = point_cloud.points - mean
        u, s, vh = np.linalg.svd(point_cloud_centered, full_matrices=False)
        transformation = np.eye(4)
        transformation[:3, :3] = vh
        transformation[:3, 3] = mean
        return transformation

    def align_point_cloud(self, point_cloud, transformation):
        # 使用 PCA 的结果对点云进行变换
        points = np.asarray(point_cloud.points)
        # 将点扩展为齐次坐标
        points_homogeneous = np.hstack((points, np.ones((points.shape[0], 1))))
        # 应用变换
        points_transformed = np.dot(points_homogeneous, transformation.T)
        # 只保留变换后的 x, y, z 坐标
        points_transformed = points_transformed[:, :3]
        point_cloud.points = o3d.utility.Vector3dVector(points_transformed)
        return point_cloud


    def correct_pca_orientation(self, transformation):
        # transformation 是一个4x4的变换矩阵。transformation[:3, 0] 表示提取该矩阵的第一列的前三个元素（即主轴的方向向量）。在PCA中，第一列通常代表了数据的主轴方向。
        primary_axis = transformation[:3, 0]
        #  检查 primary_axis 的Z分量（primary_axis[2]）是否为负。
        if primary_axis[2] < 0:  # 如果Z分量为负，将整个主轴向量取反。这样可以确保主轴的Z方向为正，避免因为对称性或其他原因导致的主轴方向翻转问题。
            transformation[:3, 0] = -primary_axis
        return transformation

    def compute_centroid(self, point_cloud):
        #计算质心
        points = np.asarray(point_cloud.points)
        centroid = np.mean(points, axis=0)
        return centroid

    def translate_point_cloud(self, point_cloud, vector):
        #平移向量
        translated_points = np.asarray(point_cloud.points) + vector
        point_cloud.points = o3d.utility.Vector3dVector(translated_points)
        return point_cloud
    def combine_transformations(self, coarse_transformation_source, coarse_transformation_target, translation_vector):
        # 计算目标点云变换矩阵的逆
        inverse_transformation_target = np.linalg.inv(coarse_transformation_target)
        # 创建包含平移矢量的变换矩阵
        translation_matrix = np.eye(4)
        translation_matrix[:3, 3] = translation_vector
        # 组合变换矩阵
        combined_transformation = np.dot(translation_matrix, 
                                     np.dot(inverse_transformation_target, 
                                            coarse_transformation_source))
        return combined_transformation
    # 计算目标点云变换矩阵的逆
    def pca_coarse_registration(self, source, target, valsource, valtarget, threshold):
        # 应用 PCA 进行配准
        start_time_pca= time.time()
        transformation_source = self.apply_pca(source)
        # transformation_source = self.correct_pca_orientation(transformation_source) #主轴校正
        transformation_target = self.apply_pca(target)
        # transformation_target = self.correct_pca_orientation(transformation_target) #主轴校正
        source_aligned = self.align_point_cloud(source, transformation_source)
        target_aligned = self.align_point_cloud(target, transformation_target)
        #计算平移向量
        centroid_source = self.compute_centroid(source_aligned)
        centroid_target = self.compute_centroid(target_aligned)
        translation_vector = centroid_target - centroid_source
        source_aligned = self.translate_point_cloud(source_aligned, translation_vector)

        end_time_pca = time.time()
        pca_time = end_time_pca - start_time_pca

        print(f"PCA source 结果：\n{transformation_source}")
        print(f"PCA target 结果：\n{transformation_target}")
        print(f"平移向量：\n{translation_vector}")
        print(f"pca粗配准耗时: {pca_time} 秒")


        # 评估粗配准结果
        fitness, inlier_rmse = evaluate_registration(source_aligned, target_aligned, np.eye(4), threshold)
        print("粗配准后的评估结果：")
        print(f"RMSE: {inlier_rmse}")
        print(f"Fitness: {fitness}")
        # visualize_initial_point_clouds(source_aligned, target_aligned, window_name='粗配准结果')
        valsource_aligned, valtarget_aligned = None, None
        # valsource_aligned = self.align_point_cloud(valsource, transformation_source)
        # valsource_aligned = self.translate_point_cloud(valsource, translation_vector)
        # valtarget_aligned = self.align_point_cloud(valtarget, transformation_target)
        # visualize_initial_point_clouds(valsource_aligned, valtarget_aligned, window_name='验证粗配准结果')
        return source_aligned, target_aligned, valsource_aligned, valtarget_aligned, transformation_source, transformation_target, translation_vector




In [4]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

# 步骤 1: 读取点云
def load_point_cloud(filename):
    return np.loadtxt(filename)

# 步骤 2: 计算曲率
def compute_curvature(points, k=30):
    nbrs = NearestNeighbors(n_neighbors=k+1, algorithm='auto').fit(points)
    _, indices = nbrs.kneighbors(points)
    
    curvatures = []
    for i, point in enumerate(points):
        neighbors = points[indices[i, 1:]]  # 排除自身
        centroid = np.mean(neighbors, axis=0)
        cov_matrix = np.cov((neighbors - centroid).T)
        eigvals, eigvecs = np.linalg.eigh(cov_matrix)
        curvature = eigvals[0] / np.sum(eigvals)
        curvatures.append(curvature)
    
    return np.array(curvatures)

# 步骤 3: 找到曲率最高的边缘点
def extract_high_curvature_points(points, curvatures, num_points):
    top_curv_indices = np.argsort(curvatures)[-num_points:]
    return points[top_curv_indices]

# 主函数
def main():
    filename = "/home/daichang/Desktop/ros2_ws/src/markless-calibration/pcd/wait-to-reg/newteeth_segtop.txt"
    output_filename = "/home/daichang/Desktop/ros2_ws/src/markless-calibration/pcd/wait-to-reg/extracted_edge_points.txt"
    num_points = 300  # 选择曲率最高的300个点
    
    points = load_point_cloud(filename)
    curvatures = compute_curvature(points)
    high_curvature_points = extract_high_curvature_points(points, curvatures, num_points)
    
    print("Extracted high curvature points:")
    print(high_curvature_points)

    # 保存提取的边缘点到新文件
    np.savetxt(output_filename, high_curvature_points, fmt='%f', delimiter=' ')

if __name__ == "__main__":
    main()


Extracted high curvature points:
[[-0.039757   -0.054146    0.28349999]
 [-0.034907   -0.052452    0.28496999]
 [-0.028922   -0.04502     0.28327301]
 [-0.035797   -0.052694    0.28394201]
 [-0.030089   -0.043709    0.283555  ]
 [-0.031698   -0.048831    0.28413999]
 [-0.032409   -0.048414    0.28408301]
 [-0.028945   -0.044976    0.28322199]
 [-0.029094   -0.044153    0.28325099]
 [-0.030692   -0.04835     0.28454399]
 [-0.040503   -0.054699    0.28493801]
 [-0.029121   -0.044124    0.28323501]
 [-0.029279   -0.044102    0.28310099]
 [-0.030589   -0.031528    0.28173399]
 [-0.030134   -0.043262    0.28340301]
 [-0.031435   -0.048471    0.283847  ]
 [-0.030039   -0.043359    0.28336301]
 [-0.037732   -0.052827    0.28345099]
 [-0.030301   -0.04366     0.283732  ]
 [-0.029913   -0.043394    0.28309301]
 [-0.029254   -0.044078    0.28312799]
 [-0.028695   -0.038254    0.282695  ]
 [-0.036295   -0.051238    0.28481001]
 [-0.036285   -0.051259    0.28475699]
 [-0.03007    -0.031269    0.28